<a href="https://colab.research.google.com/github/srv96/AI-ML-TensorFlow/blob/main/Mirrored_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
cifar10 = tf.keras.datasets.cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train , X_test = X_train / 255.0 , X_test / 255.0
y_train , y_test = y_train.flatten() , y_test.flatten()

print("X_train.shape : ", X_train.shape)
print("X_test.shape : ", X_test.shape)
print("y_train.shape : ", y_train.shape)
print("y_test.shape : ", y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step
X_train.shape :  (50000, 32, 32, 3)
X_test.shape :  (10000, 32, 32, 3)
y_train.shape :  (50000,)
y_test.shape :  (10000,)


In [4]:
K = len(set(y_train))
print("number of classes : ",K)

number of classes :  10


In [5]:
def create_model():
  i = tf.keras.layers.Input(shape=X_train[0].shape)

  x = tf.keras.layers.Conv2D(32, (3,3), activation='relu' , padding = 'same')(i)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2D(32, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.MaxPool2D((2,2))(x)

  x = tf.keras.layers.Conv2D(64, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2D(64, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.MaxPool2D((2,2))(x)

  x = tf.keras.layers.Conv2D(128, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2D(128, (3,3), activation='relu' , padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.MaxPool2D((2,2))(x)

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(1024, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(K, activation='softmax')(x)

  model = tf.keras.Model(i,x)
  return model

In [6]:
strategy = tf.distribute.MirroredStrategy()

In [7]:
print(f'Nmuber of devices : {strategy.num_replicas_in_sync}')

Nmuber of devices : 1


In [8]:
with strategy.scope():
  model = create_model()

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


In [9]:
r = model.fit(
    X_train,y_train,
    validation_data = (X_test,y_test),
    batch_size = 128,
    epochs = 15
)

Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.4188 - loss: 2.0363 - val_accuracy: 0.2791 - val_loss: 2.2729
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6747 - loss: 0.9168 - val_accuracy: 0.7119 - val_loss: 0.8279
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7603 - loss: 0.6828 - val_accuracy: 0.7349 - val_loss: 0.7584
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8063 - loss: 0.5521 - val_accuracy: 0.7624 - val_loss: 0.6857
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8443 - loss: 0.4466 - val_accuracy: 0.7803 - val_loss: 0.6523
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8754 - loss: 0.3544 - val_accuracy: 0.7857 - val_loss: 0.6768
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8969 - loss: 0.2947 - val_accuracy: 0.7941 - val_loss: 0.6813
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9153 - loss: 0.2372 - val_acc

In [10]:
#without distribution
model2 = create_model()

model2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

r = model2.fit(
    X_train,y_train,
    validation_data = (X_test,y_test),
    batch_size = 128,
    epochs = 15
)

Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 23s 31ms/step - accuracy: 0.4159 - loss: 2.1245 - val_accuracy: 0.2671 - val_loss: 2.4958
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6835 - loss: 0.8977 - val_accuracy: 0.6928 - val_loss: 0.8623
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7669 - loss: 0.6703 - val_accuracy: 0.7343 - val_loss: 0.7715
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8129 - loss: 0.5393 - val_accuracy: 0.7758 - val_loss: 0.6653
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8427 - loss: 0.4485 - val_accuracy: 0.7911 - val_loss: 0.6296
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8721 - loss: 0.3681 - val_accuracy: 0.7856 - val_loss: 0.6493
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8957 - loss: 0.2940 - val_accuracy: 0.7709 - val_loss: 0.7129
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9124 - loss: 0.2481 - val_accuracy: 